In [ ]:
import pandas as pd
from transformers import pipeline
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
import re

/workspace/.pyenv_mirror/user/current/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
labels = ['information','appreciation','complain']
df = pd.read_csv(r'sg_text_sent - pairs.csv')
df2 = pd.read_excel(r'Comments.xlsx')

df.head()

Downloading: 100%|██████████| 1.15k/1.15k [00:00<00:00, 577kB/s]
Downloading: 100%|██████████| 1.63G/1.63G [00:31<00:00, 51.9MB/s]
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 17.4kB/s]
Downloading: 100%|██████████| 899k/899k [00:00<00:00, 2.40MB/s]
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 1.82MB/s]
Downloading: 100%|██████████| 1.36M/1.36M [00:00<00:00, 3.13MB/s]


,instance,related,sen_score,stem
0,experience,suffering bad,-1.75,suffering bad
1,app,worthless capture,-1.60,worthless captur
2,experience,cancelled pathetic,-1.55,cancelled pathet
3,app,bogus fraud,-1.50,bogus fraud
4,service,disappointed poor,-1.50,disappointed poor


In [16]:
df['comments'] = None
for i in range(len(df)):
    for j in df2['Comments']:
        if isinstance(j, str):
            if df['instance'][i] in j and df['related'][i] in j:
                df['comments'][i] = j

/tmp/ipykernel_2520/920172940.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comments'][i] = j


None
None
None
None
None
Very poor disgusting service. Took so long time. Didnâ€™t address my problem. If cannot provide good service then STOP IT
None
None
None
None
None
None
None
None
very unstable app i lost money
Customer service executive was good. But, abruptly cut the call. However the issue is with NeuCoins. Itâ€™s advertised as 5% across the app but has hidden terms and conditions. Thatâ€™s misleading
None
Disagree, I have booked philips 3000 series dryer 3days before still order is not yet confirmed, really disappointed..
None
You have launched the app in a hurry, there are lot of technical glitches and customers do not have the facility of choosing options in filters like list of products available under qik emi card and FAQ are not at all usefull. There is no clarity about what is what exactly defined.
None
None
Nothing is resolved at all. If I am mot able to use UPI, this app is worthless for me and only capture space on my phone.
None
I received only FALSE assurances and

In [4]:
labels = ['information','appreciation','complain']
df = pd.read_csv(r'with_comments.csv')
df['label'] = None
for i in range(len(df)):
    if isinstance(df['comments'][i], str):
        label = classifier(df["comments"][i][0:500],labels)
        df['label'][i] = label['labels'][0]
        print(i)

/tmp/ipykernel_1595/3646420827.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = label['labels'][0]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27